# Feature Engineering e Dados Curados

Este notebook tem como propósito realizar a engenharia de atributos completa a partir dos dados processados, garantindo que o conjunto final de variáveis seja coerente, numerificado e adequado para aprendizado supervisionado. Serão aplicadas as transformações necessárias para resolver problemas de cardinalidade, remover colunas irrelevantes e assegurar que treino e teste estejam alinhados de forma robusta. 

Ao final, os arquivos serão salvos em `data/curated/`, prontos para rastreamento, versionamento e uso em experimentos de modelagem.


## Diagnóstico de Cardinalidade

Esta etapa tem como objetivo analisar todas as colunas não numéricas presentes nos dados de treino e teste, identificando quantos valores únicos cada uma possui. O foco é antecipar possíveis problemas de alta cardinalidade que possam gerar sparsidade excessiva ou explosão de features no momento da codificação.

Será criado um relatório completo com a cardinalidade de cada variável categórica, permitindo decisões fundamentadas sobre agrupamento, transformação ou exclusão. O resultado final desta célula técnica é um arquivo `.csv` salvo em `data/curated/`, que servirá como referência rastreável para as próximas etapas de feature engineering.


In [5]:
# ETAPA: Diagnóstico de Cardinalidade

"""
Esta etapa carrega os datasets processados, identifica todas as colunas não numéricas
e calcula a cardinalidade de cada uma, criando um relatório detalhado.
O objetivo é mapear variáveis com alta cardinalidade que precisem de agrupamento ou tratamento específico.
O relatório final será salvo em 'data/curated/cardinality_report.csv' para rastreabilidade.
"""

import os
import pandas as pd
from tqdm import tqdm

# 1️⃣ Validar diretório de trabalho
print("Current Working Directory:", os.getcwd())

# 2️⃣ Definir paths
TRAIN_PATH = 'data/processed/train_clean.csv'
TEST_PATH = 'data/processed/test_clean.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 4️⃣ Identificar colunas não numéricas
train_non_numeric = train_df.select_dtypes(include=['object']).columns.tolist()
test_non_numeric = test_df.select_dtypes(include=['object']).columns.tolist()
print("\nColunas não numéricas (treino):", train_non_numeric)
print("Colunas não numéricas (teste):", test_non_numeric)

# 5️⃣ Diagnóstico de cardinalidade
cardinality_info = []

print("\nCalculando cardinalidade por coluna com barra de progresso...")

for col in tqdm(train_non_numeric, desc="Treino"):
    unique_vals = train_df[col].nunique(dropna=True)
    cardinality_info.append({'Set': 'Train', 'Column': col, 'UniqueValues': unique_vals})

for col in tqdm(test_non_numeric, desc="Teste"):
    unique_vals = test_df[col].nunique(dropna=True)
    cardinality_info.append({'Set': 'Test', 'Column': col, 'UniqueValues': unique_vals})

# 6️⃣ Criar DataFrame de relatório
cardinality_df = pd.DataFrame(cardinality_info)

print("\nResumo das 20 primeiras linhas:")
print(cardinality_df.head(20))

# 7️⃣ Salvar relatório para rastreabilidade
os.makedirs('data/curated', exist_ok=True)
cardinality_df.to_csv('data/curated/cardinality_report.csv', index=False)
print("\nRelatório salvo em: data/curated/cardinality_report.csv")


Current Working Directory: /workspace

Colunas não numéricas (treino): ['ID', 'Customer_ID', 'Month', 'Name', 'SSN', 'Occupation', 'Num_of_Loan', 'Type_of_Loan', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Credit_Mix', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Payment_Behaviour', 'Credit_Score']
Colunas não numéricas (teste): ['Num_of_Loan', 'Type_of_Loan', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Credit_History_Age', 'Payment_of_Min_Amount']

Calculando cardinalidade por coluna com barra de progresso...


Teste: 100%|██████████| 6/6 [00:00<00:00, 900.84it/s]


Resumo das 20 primeiras linhas:
      Set                  Column  UniqueValues
0   Train                      ID        100000
1   Train             Customer_ID         12500
2   Train                   Month             8
3   Train                    Name         10139
4   Train                     SSN         12501
5   Train              Occupation            16
6   Train             Num_of_Loan           434
7   Train            Type_of_Loan          6260
8   Train  Num_of_Delayed_Payment           749
9   Train    Changed_Credit_Limit          4384
10  Train              Credit_Mix             4
11  Train      Credit_History_Age           404
12  Train   Payment_of_Min_Amount             3
13  Train       Payment_Behaviour             7
14  Train            Credit_Score             3
15   Test             Num_of_Loan           263
16   Test            Type_of_Loan          6260
17   Test  Num_of_Delayed_Payment           443
18   Test    Changed_Credit_Limit          3927
19   Te

## Tratamento de Alta Cardinalidade

Esta etapa tem como objetivo reduzir a dimensionalidade excessiva causada por variáveis categóricas com muitos valores únicos. Serão removidas colunas que são identificadores diretos e não contribuem para aprendizado supervisionado. Além disso, colunas com cardinalidade muito alta serão agrupadas ou transformadas em faixas, quando aplicável, para evitar sparsidade excessiva após a codificação.

O resultado esperado é um conjunto de dados mais compacto, coerente e alinhado entre treino e teste, garantindo que o processamento posterior gere um número controlado de features, mantendo a capacidade preditiva do modelo.


## Coerção de Tipos, Binning Seguro e Padronização

Esta etapa executa de forma consolidada a coerção de tipos para garantir que colunas com dados inconsistentes possam ser transformadas em faixas numéricas de forma segura. Além disso, realiza a padronização da coluna 'Month' apenas quando ela estiver presente, evitando falhas no pipeline. O resultado esperado é um conjunto de dados limpo, com variáveis agrupadas ou convertidas para reduzir a cardinalidade, mantendo a coerência entre treino e teste.


In [8]:
# 🔧 ETAPA: Coerção de Tipos, Binning Seguro e Padronização de 'Month'

"""
Executa:
1) Exclusão de identificadores.
2) Coerção de tipos numéricos com tratamento de strings.
3) Binning controlado para alta cardinalidade.
4) Padronização segura da coluna 'Month'.
Fluxo rastreável, alinhado ao PROTOCOLO V5.4.
"""

import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# 1️⃣ Validar CWD
print("Current Working Directory:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/processed/train_clean.csv'
TEST_PATH = 'data/processed/test_clean.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 4️⃣ Remover identificadores
drop_cols = ['ID', 'Customer_ID', 'Name', 'SSN']
train_df.drop(columns=drop_cols, errors='ignore', inplace=True)
test_df.drop(columns=drop_cols, errors='ignore', inplace=True)

print("\nColunas removidas:", drop_cols)

# 5️⃣ Coerção de tipos e binning para Num_of_Loan
if 'Num_of_Loan' in train_df.columns:
    for df in [train_df, test_df]:
        df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')

    for df in [train_df, test_df]:
        df['Num_of_Loan_Bin'] = pd.cut(
            df['Num_of_Loan'],
            bins=[0,1,2,5,10,20,50,100,500,1000],
            labels=False
        )

    train_df.drop(columns=['Num_of_Loan'], inplace=True)
    test_df.drop(columns=['Num_of_Loan'], inplace=True)
    print("\nBinning aplicado: Num_of_Loan")

# 6️⃣ Outras colunas de alta cardinalidade
for col in ['Changed_Credit_Limit', 'Num_of_Delayed_Payment', 'Credit_History_Age']:
    if col in train_df.columns:
        for df in [train_df, test_df]:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col + '_Bin'] = pd.qcut(
                df[col].rank(method="first"),
                q=10 if col != 'Credit_History_Age' else 5,
                labels=False,
                duplicates='drop'
            )
        train_df.drop(columns=[col], inplace=True)
        test_df.drop(columns=[col], inplace=True)
        print(f"Binning aplicado: {col}")

# 7️⃣ Padronização segura de 'Month'
month_map = {
    'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,
    'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12
}

for df_name, df in [('Treino', train_df), ('Teste', test_df)]:
    if 'Month' in df.columns:
        df['Month_Num'] = df['Month'].map(month_map)
        df.drop(columns=['Month'], inplace=True, errors='ignore')
        print(f"Coluna 'Month' padronizada e removida em: {df_name}")
    else:
        print(f"Coluna 'Month' não encontrada em: {df_name}")

print("\nColunas finais após coerção, binning e padronização:")
print(train_df.columns.tolist())

print("\nAmostra treino (head 20):")
print(train_df.head(20))

# 8️⃣ Salvar versão intermediária
os.makedirs('data/curated', exist_ok=True)
train_df.to_csv('data/curated/train_feature_engineered.csv', index=False)
test_df.to_csv('data/curated/test_feature_engineered.csv', index=False)

print("\nArquivos salvos em: data/curated/train_feature_engineered.csv e data/curated/test_feature_engineered.csv")


Current Working Directory: /workspace

Colunas removidas: ['ID', 'Customer_ID', 'Name', 'SSN']

Binning aplicado: Num_of_Loan
Binning aplicado: Changed_Credit_Limit
Binning aplicado: Num_of_Delayed_Payment
Binning aplicado: Credit_History_Age
Coluna 'Month' padronizada e removida em: Treino
Coluna 'Month' não encontrada em: Teste

Colunas finais após coerção, binning e padronização:
['Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Type_of_Loan', 'Delay_from_due_date', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score', 'Num_of_Loan_Bin', 'Changed_Credit_Limit_Bin', 'Num_of_Delayed_Payment_Bin', 'Credit_History_Age_Bin', 'Month_Num']

Amostra treino (head 20):
      Age Occupation  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0    23.0  Sc

## Codificação Final e Alinhamento dos Dados Curados

Esta etapa aplica a codificação final de variáveis categóricas usando `get_dummies()`, garantindo que todas as variáveis sejam numéricas e adequadas para modelagem supervisionada. Em seguida, alinha os conjuntos de treino e teste para que possuam exatamente as mesmas colunas, evitando inconsistências futuras. O resultado esperado é o salvamento dos arquivos finais `train_curated.csv` e `test_curated.csv` em `data/curated/`, prontos para rastreamento e versionamento com DVC.


In [ ]:
# 🔧 ETAPA: Codificação Final, Alinhamento e Versionamento dos Dados Curados

"""
Executa:
1) Carrega datasets feature engineered.
2) Identifica colunas não numéricas remanescentes.
3) Aplica get_dummies com drop_first para evitar multicolinearidade.
4) Alinha treino e teste.
5) Salva camada 'Curated' final.
6) Versiona com DVC.
Fluxo 100% rastreável, com tqdm e auditoria head(20).
"""

import os
import pandas as pd
from tqdm import tqdm
import subprocess

# 1️⃣ Validar CWD
print("Current Working Directory:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/curated/train_feature_engineered.csv'
TEST_PATH = 'data/curated/test_feature_engineered.csv'
CURATED_TRAIN_PATH = 'data/curated/train_curated.csv'
CURATED_TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 4️⃣ Identificar colunas não numéricas
train_non_numeric = train_df.select_dtypes(include=['object']).columns.tolist()
test_non_numeric = test_df.select_dtypes(include=['object']).columns.tolist()
print("\nColunas não numéricas (treino):", train_non_numeric)
print("Colunas não numéricas (teste):", test_non_numeric)

# 5️⃣ Codificação com tqdm
print("\nAplicando get_dummies...")
train_encoded = pd.get_dummies(train_df, drop_first=True)
test_encoded = pd.get_dummies(test_df, drop_first=True)

# 6️⃣ Alinhar colunas
print("\nAlinhando treino e teste...")
with tqdm(total=1, desc="Alinhamento") as pbar:
    train_aligned, test_aligned = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)
    pbar.update(1)

# 7️⃣ Prints de auditoria
print("\nTreino head(20):")
print(train_aligned.head(20))
print("\nTeste head(20):")
print(test_aligned.head(20))

# 8️⃣ Salvar versão final 'Curated'
train_aligned.to_csv(CURATED_TRAIN_PATH, index=False)
test_aligned.to_csv(CURATED_TEST_PATH, index=False)
print(f"\nArquivos salvos: {CURATED_TRAIN_PATH} | {CURATED_TEST_PATH}")

# 9️⃣ Versionamento com DVC
print("\nVersionando com DVC...")
with tqdm(total=3, desc="DVC Pipeline") as pbar:
    subprocess.run(['dvc', 'add', CURATED_TRAIN_PATH])
    pbar.update(1)
    subprocess.run(['dvc', 'add', CURATED_TEST_PATH])
    pbar.update(1)
    subprocess.run(['git', 'add', CURATED_TRAIN_PATH + '.dvc', CURATED_TEST_PATH + '.dvc'])
    subprocess.run(['git', 'commit', '-m', 'Add final curated train and test datasets'])
    subprocess.run(['dvc', 'push'])
    pbar.update(1)

print("\nCamada 'Curated' final criada, alinhada, versionada e enviada ao backend remoto.")


Current Working Directory: /workspace

Colunas não numéricas (treino): ['Occupation', 'Type_of_Loan', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour', 'Credit_Score']
Colunas não numéricas (teste): ['Type_of_Loan', 'Payment_of_Min_Amount']

Aplicando get_dummies...

Alinhando treino e teste...


Alinhamento: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]



Treino head(20):
      Age  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0    23.0       19114.12            1824.843333                  3   
1    23.0       19114.12                    NaN                  3   
2  -500.0       19114.12                    NaN                  3   
3    23.0       19114.12                    NaN                  3   
4    23.0       19114.12            1824.843333                  3   
5    23.0       19114.12                    NaN                  3   
6    23.0       19114.12            1824.843333                  3   
7    23.0       19114.12            1824.843333                  3   
8    33.0       34847.84            3037.986667                  2   
9    28.0       34847.84            3037.986667                  2   
10   28.0       37550.74            3037.986667                  2   
11   28.0       34847.84                    NaN                  2   
12   28.0       34847.84            3037.986667                  2   
13

DVC Pipeline:  67%|██████▋   | 2/3 [00:20<00:09,  9.28s/it]

[main 0c38ca0] Add final curated train and test datasets
 2 files changed, 10 insertions(+)
 create mode 100644 data/curated/test_curated.csv.dvc
 create mode 100644 data/curated/train_curated.csv.dvc


## Auditoria Final dos Arquivos Curated

Esta etapa confirma o tamanho real dos arquivos `train_curated.csv` e `test_curated.csv` salvos na camada `Curated`. Além disso, apresenta informações detalhadas de linhas, colunas e tipos de dados para verificar consistência antes de executar qualquer `dvc push` novamente. O objetivo é garantir rastreabilidade total do pipeline e evitar duplicidade de operações no backend.


In [1]:
## ETAPA: Auditoria Final — Tamanho e Estrutura dos Arquivos Curated

"""
Verifica:
1) Existência e tamanho em bytes/MB dos arquivos.
2) Dimensões (linhas x colunas).
3) Tipos de dados para garantir coerência.
"""

import os
import pandas as pd

# 1️⃣ Definir paths
CURATED_TRAIN_PATH = 'data/curated/train_curated.csv'
CURATED_TEST_PATH = 'data/curated/test_curated.csv'

# 2️⃣ Verificar tamanho dos arquivos
def file_size(path):
    size_bytes = os.path.getsize(path)
    size_mb = size_bytes / (1024 * 1024)
    return f"{size_mb:.2f} MB"

print(f"Tamanho train_curated.csv: {file_size(CURATED_TRAIN_PATH)}")
print(f"Tamanho test_curated.csv:  {file_size(CURATED_TEST_PATH)}")

# 3️⃣ Carregar e mostrar estrutura resumida
train_df = pd.read_csv(CURATED_TRAIN_PATH, nrows=100)  # carrega parcial para info
test_df = pd.read_csv(CURATED_TEST_PATH, nrows=100)

print("\nTreino — Amostra:")
print(f"Linhas (estimado pelo arquivo): variável")
print(f"Colunas: {len(train_df.columns)}")
print(train_df.info())

print("\nTeste — Amostra:")
print(f"Linhas (estimado pelo arquivo): variável")
print(f"Colunas: {len(test_df.columns)}")
print(test_df.info())


Tamanho train_curated.csv: 3610.34 MB
Tamanho test_curated.csv:  1805.16 MB

Treino — Amostra:
Linhas (estimado pelo arquivo): variável
Colunas: 6305
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 6305 entries, Age to Credit_Score_Standard
dtypes: bool(6287), float64(13), int64(5)
memory usage: 628.2 KB
None

Teste — Amostra:
Linhas (estimado pelo arquivo): variável
Colunas: 6305
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 6305 entries, Age to Credit_Score_Standard
dtypes: bool(6285), float64(13), int64(7)
memory usage: 629.5 KB
None


---
Refazendo Feature Engineering após estouro de Kernel por alta cardinalidade
---

---